Importing packages

In [5]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

import warnings
warnings.filterwarnings('ignore')

Load the pretrained Network

In [6]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [7]:
model_vgg16_conv.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

Freeze the layers

In [8]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

Training parameters

In [9]:
from google.colab import drive
drive.mount('/content/drive')
img_width, img_height = 150, 150
train_data_dir = '/content/drive/My Drive/Colab Notebooks/CNN transfer - Cat vs Dog/train'
val_data_dir = '/content/drive/My Drive/Colab Notebooks/CNN transfer - Cat vs Dog/Validation'
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples =11
nb_val_samples = 11
nb_epochs = 5

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Build a classification model on top of Base Network

In [10]:
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(inputs=input, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 64)                524352    
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 15239170 (58.13 MB)
Trainable params: 524482 (2.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


Dataset Preparation

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height),
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height),
                                                        batch_size=4,class_mode='categorical')

Found 22 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


Training

In [13]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks,steps_per_epoch=nb_train_samples,
                              epochs=nb_epochs, validation_data=validation_generator, validation_steps=nb_val_samples)

print('Training Completed!')

Epoch 1/5
 6/11 [===============>..............] - ETA: 8s - loss: 1.2095 - accuracy: 0.5909

11/11 [==============================] - 29s 2s/step - loss: 1.2095 - accuracy: 0.5909 - val_loss: 0.7481 - val_accuracy: 0.5455
Training Completed!


Test the model

In [14]:
img_path ='/content/drive/My Drive/Colab Notebooks/CNN transfer - Cat vs Dog/1.jpg'
label = ['Cat','Dog']
img = image.load_img(img_path,target_size=(150,150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
ind = np.where(features == 1)[1][0]
print('Predicted Array:',features)
print('Predicted Label:',label[ind])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 [==============================] - 0s 457ms/step
Predicted Array: [[2.920054e-37 1.000000e+00]]
Predicted Label: Dog


Models for image classification with weights trained on ImageNet:
Xception,
VGG16,
VGG19,
ResNet50,
InceptionV3,
InceptionResNetV2,
MobileNet,
DenseNet,
NASNet,
